In [ ]:
!pip install -r requirements.txt

In [ ]:
!python setup.py clean --all install

In [ ]:
!git clone https://github.com/akTwelve/Mask_RCNN.git

In [ ]:
!pwd

In [ ]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Root directory of the project
#ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
#sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

#sys.path.append('./samples/cork')
import corks

#sys.path.append(ROOT_DIR)

%matplotlib inline 


In [ ]:
!python3 corks.py train --dataset='./datasets/cork' --weights=coco

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs --bind_all

In [ ]:
from tensorboard import notebook
notebook.list()

In [ ]:
notebook.display(port=6006, height=1000)

# **Evaluation**

In [ ]:
import os
import sys
import json
import datetime
import time
import numpy as np
import skimage.draw
from mrcnn.config import Config
from mrcnn import utils
from mrcnn.model import log
from mrcnn import model as modellib, utils
from corks import CorkDataset
from mrcnn import visualize
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
if os.path.abspath(os.getcwd()) != '/tf/notebooks/hdd_1/sen/MyCode/Mask_RCNN_TF2':
    ROOT_DIR = os.path.abspath("./")

    # Import Mask RCNN
    sys.path.append(ROOT_DIR)  # To find local version of the library

# Root directory of the project

class InferenceConfig(corks.CorkConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
config = InferenceConfig()

config.DETECTION_MIN_CONFIDENCE = 0.7

model = modellib.MaskRCNN(mode="inference", config=config, model_dir='./logs')
# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = './logs/cork20210429T1106_all_aug_withLR0001/mask_rcnn_cork_0060.h5'
#model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

dataset_val = CorkDataset()
dataset_val.load_cork('./datasets/cork', "val")
dataset_val.prepare()

print("Running evaluation on test images.")

mrgData={}
t_prediction = 0
t_start = time.time()

for i, image_id in enumerate(dataset_val.image_ids):
        # Load image and run detection
        image = dataset_val.load_image(image_id)
        original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, config, image_id)
        #log("original_image", original_image)
        #log("image_meta", image_meta)
        #log("gt_class_id", gt_class_id)
        #log("gt_bbox", gt_bbox)
        #log("gt_mask", gt_mask)

        fPath = dataset_val.image_info[image_id]['path']
        words = fPath.split('/')
        fname = words[-1]
        fnameGT='GT'+str(i)+fname
        fnameP='P'+str(i)+fname
        visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, dataset_val.class_names, title=fnameGT, figsize=(8, 8))
        
        
        #plt.savefig(os.path.join('GroundT',fname),bbox_inches='tight')
        print("Image File Name", fname)
        # Detect objects
        t = time.time()
        r = model.detect([image], verbose=0)[0]
        
        t_prediction += (time.time() - t)
        print("Pred Time:", (time.time() - t))
        fname_json=fname.split('.')[0] + '.json'
        print(fname_json)
        visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], title=fnameP, ax=get_ax())
        #plt.savefig(os.path.join('Prediction',fname),bbox_inches='tight')
        r['masks']=r['masks'].astype(int).tolist()
        r['rois']=r['rois'].tolist()
        r['class_ids']=r['class_ids'].tolist()
        r['scores']=r['scores'].tolist()
  
        #with open(os.path.join('json_predictions',fname_json), 'w') as outfile:
        #    json.dump(r, outfile)
        

print("Prediction time: {}. Average {}/image".format(t_prediction, t_prediction / len(dataset_val.image_ids)))
print("Total time: ", time.time() - t_start)

# Testing

In [ ]:
if os.path.abspath(os.getcwd()) != '/tf/notebooks/hdd_1/sen/MyCode/Mask_RCNN_TF2':
    ROOT_DIR = os.path.abspath("./")

    # Import Mask RCNN
    sys.path.append(ROOT_DIR)  # To find local version of the library




# Root directory of the project

class InferenceConfig(corks.CorkConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
config = InferenceConfig()

model = modellib.MaskRCNN(mode="inference", config=config, model_dir='./logs')
# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

path = './datasets/cork/test'
files = sorted(os.listdir(path))

t_prediction = 0
t_start = time.time()

for i, file in enumerate(files):
    img_path = os.path.join(path,file)
    
    print(file)
    print(i)
    image = skimage.io.imread(img_path)
    t = time.time()
    r = model.detect([image], verbose=0)[0]
    t_prediction += (time.time() - t)
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'])

print("Number of test cases:", len(files))        
print("Prediction time: {}. Average {}/image".format(t_prediction, t_prediction / len(files)))
print("Total time: ", time.time() - t_start)

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
#image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
AP_Range=[]
A_F1 = []
A_Prec = []
A_Rec = []
iou_scores=[]
totalTP = 0
totalFP = 0
totalFN = 0
for i,image_id in enumerate(dataset_val.image_ids):
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, config,
                               image_id)
    #molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
    # Run object detection
    t = time.time()
    results = model.detect([image], verbose=0)
    pred_time = time.time() - t
    r = results[0]
    # Compute AP
    gt_match,pred_match,overlap = utils.compute_matches(gt_bbox, gt_class_id, gt_mask,
        r["rois"], r["class_ids"], r["scores"], r['masks'])
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    
    AP_rng =utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    
    if math.isnan(AP_rng):
        AP_rng=0
    print("AP RANGE",AP_rng)
    prec, rec, f1 = utils.compute_f1_score(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    
    fPath = dataset_val.image_info[image_id]['path']
    words = fPath.split('/')
    fname = words[4]
    FP = np.count_nonzero(pred_match == -1)
    TP = len(pred_match) - FP
    FN = np.count_nonzero(gt_match == -1)
    print("File Name: ", fname + "-----------------------------")
    #print("GT",gt_match)
    #print("PRED",pred_match)
    if FP == 0 and TP == 0 and FN == 0:
      AP = 1.0
    elif len(gt_match)==0 and FP > 0:
      AP = 0.0
    elif len(pred_match)==0 and FN > 0:
      AP = 0.0
    #print("AP", AP)
    
    iou=utils.compute_overlaps_masks(gt_mask,r['masks'])
    if len(iou)==0:
        iou=0
    else:
        iou=np.sum(iou,axis=1)
        iou=np.mean(iou)
        
    iou_scores.append(iou)
    
    print("Prediction Time:",pred_time)
    print("False Positive:",FP)
    print("True Positive:",TP)
    print("False Negative:",FN)
    print("Precision",prec)
    print("Recall",rec)
    print("F1-score",f1)
    print("IoU Score",iou)
    print("AP",AP)
    #try:
    #    p = TP / (TP + FP)
    #    r = TP / (TP + FN)
    #except:
    #    if FP == 0 and TP == 0 and FN == 0:
    #        p = 1
    #        r = 1
    #    else:
    #        p = 0
    #        r = 1
    #    pass
    #print("PRECISION MAN",p)
    #print("RECALL MAN",r)
    
    totalTP = totalTP + TP
    totalFP = totalFP + FP
    totalFN = totalFN + FN
    print("####################################################")
    APs.append(AP)
    AP_Range.append(AP_rng)
    A_F1.append(f1)
    A_Prec.append(prec)
    A_Rec.append(rec)
    
print("Summary:-")
print("Confidence Threshold:", config.DETECTION_MIN_CONFIDENCE)
print("Total True Positive:", totalTP)
print("Total False Positive:", totalFP)
print("Total False Negative:", totalFN)
print("Average Recall:", np.mean(A_Rec))
print("Average Precision:", np.mean(A_Prec))
print("Mean F1-Score:", np.mean(A_F1))
print("Average IoU Score",np.mean(iou_scores))
print("mAP@0.5: ", np.mean(APs))
print("mAP@0.5-0.95: ", np.mean(AP_Range))

In [ ]:
!nvidia-smi